In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
# Extract and parse HTML

def decision_number_html_extractor(url):
    # Extracts the HTML with the decision numbers
    response = requests.get(url)
    html = BeautifulSoup(response.text, "html.parser")
    details = html.find_all("div", class_="field field--name-field-product-number field--type-string field--label-hidden field__item")
    return(details)
    
def html_splitter(html_output, results):
    # Splits the decision number out of the HTML
    html_ouput = html_output
    
    for i in range(0, len(html_output)):
        output_string = str(html_ouput[i])
        step1 = output_string.split(">",1)[1]
        step2 = step1.split("</div>",1)[0]
        results.append(step2)
    
    return(results)

def new_url_generator(base_url, decision_no):
    # Creates a new URL with decision numbers
    new_list = str(decision_no).split(",")
    addition = "%2C".join(new_list).lower()
    new_url = base_url + addition
    return(new_url)

In [ ]:
recursive_url = "https://www.gao.gov/search?f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01%2Bend%2B2020-01-01&sort_by=docdate&sort_order=DESC&keyword=&f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01%2Bend%2B2020-01-01&page="

results = []

for p in range(0, 543):
    new_url = recursive_url+str(p)
    html = decision_number_html_extractor(new_url)
    html_splitter(html, results)

In [ ]:
base_url_input = "https://www.gao.gov/products/"

aggregate_results = {}

for d in range(0, len(results)):
    url = new_url_generator(base_url_input, results[d])
    response = requests.get(url)
    html = BeautifulSoup(response.text, "html.parser")
    status = html.find("div", class_ = "status highlighted-status")
    highlights = html.find_all("div", class_ = "clearfix text-formatted field field--name-product-highlights-custom field--type-text-long field--label-above quickedit-field")
    document = html.find_all("div", class_ = "js-endpoint-view-decision field field--name-field-html-block field--type-text-long field--label-above")
    protest_dict = {'result_no' : str(results[d]), 'status' : str(status), 'highlights' : str(highlights), "document": str(document), "url": url}
    aggregate_results[d] = protest_dict